In [182]:
import pandas as pd
import numpy as np
import talib as ta

In [183]:
data = pd.read_csv("../price_data.csv").iloc[:,1:]
data.head()

data = data.replace(',','', regex=True)
tens = dict(k=10e3, m=10e6, b=10e9)

volume_cols = [col for col in data.columns if 'Volume' in col]
for col in volume_cols:
    for elem in range(len(data[col])):
        factor, exp = data[col][elem][0:-1], data[col][elem][-1].lower()
        data[col][elem] = int(float(factor) * tens[exp])

data.head()

#set date column as datetime
data['Date'] = pd.to_datetime(data['Date'])

#reoder dataset by increasing date
data.sort_values(by=['Date'], ascending=True, inplace=True)

/var/folders/j2/7j0fctvs2258gcs69mmjmlsr0000gn/T/ipykernel_7254/2255711551.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data[col][elem] = int(float(factor) * tens[exp])


In [184]:
# Get columns with open prices
open_cols = [col for col in data.columns if "Open" in col]
high_cols = [col for col in data.columns if "High" in col]
low_cols = [col for col in data.columns if "Low" in col]
vol_cols = [col for col in data.columns if "Volume" in col]
coins = [col.split("_")[1] for col in open_cols]
returns_cols = [col for col in data.columns if "return" in col]

for elem in range(len(open_cols)):

    open = data[open_cols[elem]]
    high = data[high_cols[elem]]
    low = data[low_cols[elem]]
    vol = data[vol_cols[elem]]
    coin = coins[elem]
    
    #overlap studies indicators
    data["SMA_{}".format(coin)] = ta.SMA(open, timeperiod=7)
    data["SMA_{}".format(coin)] = ta.SMA(open, timeperiod=28)
    data["EMA_{}".format(coin)] = ta.EMA(open, timeperiod=7)
    data["EMA_{}".format(coin)] = ta.EMA(open, timeperiod=28)
    data["WMA_{}".format(coin)] = ta.WMA(open, timeperiod=7)
    data["WMA_{}".format(coin)] = ta.WMA(open, timeperiod=28)
    data["DEMA_{}".format(coin)] = ta.DEMA(open, timeperiod=5)
    data["TEMA_{}".format(coin)] = ta.TEMA(open, timeperiod=5)
    data["TRIMA_{}".format(coin)] = ta.TRIMA(open, timeperiod=5)
    data["KAMA_{}".format(coin)] = ta.KAMA(open, timeperiod=5)
    data["MAMA_{}".format(coin)] = ta.MAMA(open, fastlimit=0.5, slowlimit=0.05)[0]
    data["T3_{}".format(coin)] = ta.T3(open, timeperiod=5, vfactor=0)

    #momentum indicators
    data["ADX_{}".format(coin)] = ta.ADX(high, low, open, timeperiod=7)
    data["ADX_{}".format(coin)] = ta.ADX(high, low, open, timeperiod=28)
    data["ADXR_{}".format(coin)] = ta.ADXR(high, low, open, timeperiod=7)
    data["ADXR_{}".format(coin)] = ta.ADXR(high, low, open, timeperiod=28)
    data["APO_{}".format(coin)] = ta.APO(open, fastperiod=12, slowperiod=26, matype=0)
    data["AROONOSC_{}".format(coin)] = ta.AROONOSC(high, low, timeperiod=14)
    data["BOP_{}".format(coin)] = ta.BOP(open, high, low, open)
    data["CCI_{}".format(coin)] = ta.CCI(high, low, open, timeperiod=14)
    data["CMO_{}".format(coin)] = ta.CMO(open, timeperiod=14)
    data["DX_{}".format(coin)] = ta.DX(high, low, open, timeperiod=14)
    data["MACD_{}".format(coin)] = ta.MACD(open, fastperiod=12, slowperiod=26, signalperiod=9)[0]
    data["MFI_{}".format(coin)] = ta.MFI(high, low, open, vol, timeperiod=14)
    data["MINUS_DI_{}".format(coin)] = ta.MINUS_DI(high, low, open, timeperiod=14)

    #volatility indicators
    data["ATR_{}".format(coin)] = ta.ATR(high, low, open, timeperiod=14)
    data["NATR_{}".format(coin)] = ta.NATR(high, low, open, timeperiod=14)
    data["TRANGE_{}".format(coin)] = ta.TRANGE(high, low, open)

    #volume indicators
    data["AD_{}".format(coin)] = ta.AD(high, low, open, vol)
    data["ADOSC_{}".format(coin)] = ta.ADOSC(high, low, open, vol, fastperiod=3, slowperiod=10)
    data["OBV_{}".format(coin)] = ta.OBV(open, vol)

    #price transform
    data["AVGPRICE_{}".format(coin)] = ta.AVGPRICE(open, high, low, open)
    data["MEDPRICE_{}".format(coin)] = ta.MEDPRICE(high, low)
    data["TYPPRICE_{}".format(coin)] = ta.TYPPRICE(high, low, open)
    data["WCLPRICE_{}".format(coin)] = ta.WCLPRICE(high, low, open)

    #cycle indicators
    data["HT_DCPERIOD_{}".format(coin)] = ta.HT_DCPERIOD(open)
    data["HT_DCPHASE_{}".format(coin)] = ta.HT_DCPHASE(open)
    data["HT_PHASOR_{}".format(coin)] = ta.HT_PHASOR(open)[0]
    data["HT_SINE_{}".format(coin)] = ta.HT_SINE(open)[0]
    data["HT_TRENDMODE_{}".format(coin)] = ta.HT_TRENDMODE(open)

    #pattern recognition
    data["CDL2CROWS_{}".format(coin)] = ta.CDL2CROWS(open, high, low, open)
    data["CDL3BLACKCROWS_{}".format(coin)] = ta.CDL3BLACKCROWS(open, high, low, open)
    data["CDL3INSIDE_{}".format(coin)] = ta.CDL3INSIDE(open, high, low, open)
    data["CDL3LINESTRIKE_{}".format(coin)] = ta.CDL3LINESTRIKE(open, high, low, open)

    data["Log_RET_{}".format(coin)] = np.log(data[returns_cols[elem]] + 1)
    #get log returns columns
    log_returns_cols = [col for col in data.columns if "Log_RET" in col]

    #add previous returns
    data["1Lag_RET_{}".format(coin)] = data[log_returns_cols[elem]].shift(1)
    data["2Lag_RET_{}".format(coin)] = data[log_returns_cols[elem]].shift(2)
    data["3Lag_RET_{}".format(coin)] = data[log_returns_cols[elem]].shift(3)
    data["4Lag_RET_{}".format(coin)] = data[log_returns_cols[elem]].shift(4)
    data["5Lag_RET_{}".format(coin)] = data[log_returns_cols[elem]].shift(5)
    data["6Lag_RET_{}".format(coin)] = data[log_returns_cols[elem]].shift(6)
    data["7Lag_RET_{}".format(coin)] = data[log_returns_cols[elem]].shift(7)

    #add previous open
    data["1Lag_OPEN_{}".format(coin)] = data[open_cols[elem]].shift(1)
    data["2Lag_OPEN_{}".format(coin)] = data[open_cols[elem]].shift(2)
    data["3Lag_OPEN_{}".format(coin)] = data[open_cols[elem]].shift(3)
    data["4Lag_OPEN_{}".format(coin)] = data[open_cols[elem]].shift(4)
    data["5Lag_OPEN_{}".format(coin)] = data[open_cols[elem]].shift(5)
    data["6Lag_OPEN_{}".format(coin)] = data[open_cols[elem]].shift(6)
    data["7Lag_OPEN_{}".format(coin)] = data[open_cols[elem]].shift(7)

    

/var/folders/j2/7j0fctvs2258gcs69mmjmlsr0000gn/T/ipykernel_7254/1690404351.py:37: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  data["AROONOSC_{}".format(coin)] = ta.AROONOSC(high, low, timeperiod=14)
/var/folders/j2/7j0fctvs2258gcs69mmjmlsr0000gn/T/ipykernel_7254/1690404351.py:38: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  data["BOP_{}".format(coin)] = ta.BOP(open, high, low, open)
/var/folders/j2/7j0fctvs2258gcs69mmjmlsr0000gn/T/ipykernel_7254/1690404351.py:39: PerformanceWarning: DataFrame is highly fragmented.  This is us

In [185]:
np.shape(data)

(1416, 781)

In [186]:
#remove _USD from Open columns in data
open_cols = [col for col in data.columns if 'Open' in col]
for col in open_cols:
    data.rename(columns={col:col.replace('_USD','')}, inplace=True)
#remove _USD from returns columns in data
returns_cols = [col for col in data.columns if 'returns' in col]
for col in returns_cols:
    data.rename(columns={col:col.replace('_USD','')}, inplace=True)

vol_cols = [col for col in data.columns if 'Volume' in col]
chg_cols = [col for col in data.columns if 'Change' in col]
open_cols = [col for col in data.columns if 'Open' in col]
high_cols = [col for col in data.columns if 'High' in col]
low_cols = [col for col in data.columns if 'Low' in col]
returns_cols = [col for col in data.columns if 'returns' in col]
log_returns_cols = [col for col in data.columns if 'Log_RET' in col]

#if returns cols are positive, set to 1, else 0
for col in returns_cols:
    data[col] = np.where(data[col] > 0, 1, 0)

data.drop(vol_cols, axis=1, inplace=True)
data.drop(chg_cols, axis=1, inplace=True)
data.drop(open_cols, axis=1, inplace=True)
data.drop(high_cols, axis=1, inplace=True)
data.drop(low_cols, axis=1, inplace=True)
data.drop(returns_cols, axis=1, inplace=True)
#data.drop(log_returns_cols, axis=1, inplace=True)

In [187]:
data.to_csv("../features.csv")

In [188]:
TRAIN_SIZE = 0.9
VAL_SIZE = 0.05

n_train = int(len(data) * TRAIN_SIZE)
n_val = int(len(data) * VAL_SIZE)

df_train = data.iloc[0:n_train]
df_val = data.iloc[n_train:n_train+n_val]
df_test = data.iloc[n_train+n_val:]

df_train.to_csv("../features_train.csv")
df_val.to_csv("../features_val.csv")
df_test.to_csv("../features_test.csv")

In [189]:
np.shape(df_test)

(72, 703)

In [190]:
#reset index
data.reset_index(drop=True, inplace=True)

In [191]:
#get the row with nan values
last_row = data[data.isnull().any(axis=1)].index[-1]
last_row

81